# import

In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [11]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [12]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [13]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred = logits, y_true = labels,
                                                                   from_logits = True))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [14]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model(class version)

In [15]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.RandomNormal() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(2):
            self.model.add(dense(256,weight_init))
            self.model.add(relu())
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [16]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [17]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [18]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [19]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.4996, train_loss: 2.19312143, train_accuracy: 0.1953, test_accuracy: 0.1138
Epoch: [ 0] [    1/  468] time: 0.6302, train_loss: 2.15299106, train_accuracy: 0.1797, test_accuracy: 0.1509
Epoch: [ 0] [    2/  468] time: 0.7674, train_loss: 2.08673215, train_accuracy: 0.2891, test_accuracy: 0.2827
Epoch: [ 0] [    3/  468] time: 0.9037, train_loss: 2.02045631, train_accuracy: 0.4688, test_accuracy: 0.4358
Epoch: [ 0] [    4/  468] time: 1.0559, train_loss: 1.93375242, train_accuracy: 0.5781, test_accuracy: 0.5580
Epoch: [ 0] [    5/  468] time: 1.2016, train_loss: 1.91077757, train_accuracy: 0.5391, test_accuracy: 0.6091
Epoch: [ 0] [    6/  468] time: 1.3419, train_loss: 1.78960037, train_accuracy: 0.6641, test_accuracy: 0.6408
Epoch: [ 0] [    7/  468] time: 1.4651, train_loss: 1.72526777, train_accuracy: 0.6328, test_accuracy: 0.6895
Epoch: [ 0] [    8/  468] time: 1.5957, 

Epoch: [ 0] [   75/  468] time: 11.1419, train_loss: 0.37171131, train_accuracy: 0.8984, test_accuracy: 0.9090
Epoch: [ 0] [   76/  468] time: 11.2850, train_loss: 0.33146808, train_accuracy: 0.8906, test_accuracy: 0.9118
Epoch: [ 0] [   77/  468] time: 11.4205, train_loss: 0.24401397, train_accuracy: 0.9375, test_accuracy: 0.9125
Epoch: [ 0] [   78/  468] time: 11.5453, train_loss: 0.29809067, train_accuracy: 0.9062, test_accuracy: 0.9115
Epoch: [ 0] [   79/  468] time: 11.6690, train_loss: 0.37995327, train_accuracy: 0.8984, test_accuracy: 0.9125
Epoch: [ 0] [   80/  468] time: 11.7946, train_loss: 0.28649256, train_accuracy: 0.9062, test_accuracy: 0.9118
Epoch: [ 0] [   81/  468] time: 11.9313, train_loss: 0.42476398, train_accuracy: 0.8750, test_accuracy: 0.9123
Epoch: [ 0] [   82/  468] time: 12.0759, train_loss: 0.39287782, train_accuracy: 0.8828, test_accuracy: 0.9126
Epoch: [ 0] [   83/  468] time: 12.2235, train_loss: 0.31334633, train_accuracy: 0.9219, test_accuracy: 0.9097
E

Epoch: [ 0] [  149/  468] time: 20.8354, train_loss: 0.13539097, train_accuracy: 0.9688, test_accuracy: 0.9321
Epoch: [ 0] [  150/  468] time: 20.9591, train_loss: 0.31548855, train_accuracy: 0.9062, test_accuracy: 0.9329
Epoch: [ 0] [  151/  468] time: 21.1047, train_loss: 0.31596565, train_accuracy: 0.9219, test_accuracy: 0.9322
Epoch: [ 0] [  152/  468] time: 21.2493, train_loss: 0.22451757, train_accuracy: 0.9375, test_accuracy: 0.9304
Epoch: [ 0] [  153/  468] time: 21.3940, train_loss: 0.12192824, train_accuracy: 0.9688, test_accuracy: 0.9298
Epoch: [ 0] [  154/  468] time: 21.5346, train_loss: 0.26488170, train_accuracy: 0.9297, test_accuracy: 0.9304
Epoch: [ 0] [  155/  468] time: 21.6752, train_loss: 0.19061214, train_accuracy: 0.9531, test_accuracy: 0.9321
Epoch: [ 0] [  156/  468] time: 21.8006, train_loss: 0.23556726, train_accuracy: 0.9453, test_accuracy: 0.9312
Epoch: [ 0] [  157/  468] time: 21.9355, train_loss: 0.27867472, train_accuracy: 0.9062, test_accuracy: 0.9326
E

Epoch: [ 0] [  223/  468] time: 30.4237, train_loss: 0.15775219, train_accuracy: 0.9609, test_accuracy: 0.9418
Epoch: [ 0] [  224/  468] time: 30.5490, train_loss: 0.24322316, train_accuracy: 0.9375, test_accuracy: 0.9417
Epoch: [ 0] [  225/  468] time: 30.6694, train_loss: 0.14153239, train_accuracy: 0.9609, test_accuracy: 0.9403
Epoch: [ 0] [  226/  468] time: 30.7931, train_loss: 0.13784167, train_accuracy: 0.9609, test_accuracy: 0.9392
Epoch: [ 0] [  227/  468] time: 30.9252, train_loss: 0.11494628, train_accuracy: 0.9609, test_accuracy: 0.9375
Epoch: [ 0] [  228/  468] time: 31.0508, train_loss: 0.28965726, train_accuracy: 0.9141, test_accuracy: 0.9383
Epoch: [ 0] [  229/  468] time: 31.1771, train_loss: 0.17520338, train_accuracy: 0.9453, test_accuracy: 0.9409
Epoch: [ 0] [  230/  468] time: 31.3058, train_loss: 0.16317131, train_accuracy: 0.9453, test_accuracy: 0.9422
Epoch: [ 0] [  231/  468] time: 31.4320, train_loss: 0.11651538, train_accuracy: 0.9766, test_accuracy: 0.9430
E

Epoch: [ 0] [  297/  468] time: 40.3276, train_loss: 0.20397934, train_accuracy: 0.9531, test_accuracy: 0.9534
Epoch: [ 0] [  298/  468] time: 40.4652, train_loss: 0.16626549, train_accuracy: 0.9766, test_accuracy: 0.9531
Epoch: [ 0] [  299/  468] time: 40.6049, train_loss: 0.06550301, train_accuracy: 0.9922, test_accuracy: 0.9522
Epoch: [ 0] [  300/  468] time: 40.7365, train_loss: 0.17184061, train_accuracy: 0.9375, test_accuracy: 0.9514
Epoch: [ 0] [  301/  468] time: 40.8736, train_loss: 0.27526873, train_accuracy: 0.9141, test_accuracy: 0.9513
Epoch: [ 0] [  302/  468] time: 41.0092, train_loss: 0.11791898, train_accuracy: 0.9609, test_accuracy: 0.9514
Epoch: [ 0] [  303/  468] time: 41.1586, train_loss: 0.16358438, train_accuracy: 0.9688, test_accuracy: 0.9513
Epoch: [ 0] [  304/  468] time: 41.3283, train_loss: 0.12062728, train_accuracy: 0.9766, test_accuracy: 0.9509
Epoch: [ 0] [  305/  468] time: 41.5095, train_loss: 0.07623105, train_accuracy: 0.9922, test_accuracy: 0.9494
E

Epoch: [ 0] [  372/  468] time: 50.5735, train_loss: 0.12267962, train_accuracy: 0.9609, test_accuracy: 0.9533
Epoch: [ 0] [  373/  468] time: 50.7019, train_loss: 0.13638729, train_accuracy: 0.9453, test_accuracy: 0.9542
Epoch: [ 0] [  374/  468] time: 50.8445, train_loss: 0.09019657, train_accuracy: 0.9688, test_accuracy: 0.9552
Epoch: [ 0] [  375/  468] time: 50.9865, train_loss: 0.17532596, train_accuracy: 0.9531, test_accuracy: 0.9567
Epoch: [ 0] [  376/  468] time: 51.1431, train_loss: 0.12502368, train_accuracy: 0.9688, test_accuracy: 0.9573
Epoch: [ 0] [  377/  468] time: 51.3096, train_loss: 0.20717672, train_accuracy: 0.9453, test_accuracy: 0.9565
Epoch: [ 0] [  378/  468] time: 51.4542, train_loss: 0.18013424, train_accuracy: 0.9688, test_accuracy: 0.9558
Epoch: [ 0] [  379/  468] time: 51.6092, train_loss: 0.17981556, train_accuracy: 0.9375, test_accuracy: 0.9549
Epoch: [ 0] [  380/  468] time: 51.7475, train_loss: 0.15595242, train_accuracy: 0.9531, test_accuracy: 0.9550
E

Epoch: [ 0] [  447/  468] time: 60.9010, train_loss: 0.09650733, train_accuracy: 0.9609, test_accuracy: 0.9605
Epoch: [ 0] [  448/  468] time: 61.0227, train_loss: 0.08239527, train_accuracy: 0.9688, test_accuracy: 0.9613
Epoch: [ 0] [  449/  468] time: 61.1404, train_loss: 0.14293046, train_accuracy: 0.9688, test_accuracy: 0.9621
Epoch: [ 0] [  450/  468] time: 61.2598, train_loss: 0.13156238, train_accuracy: 0.9609, test_accuracy: 0.9619
Epoch: [ 0] [  451/  468] time: 61.3808, train_loss: 0.05899435, train_accuracy: 0.9766, test_accuracy: 0.9620
Epoch: [ 0] [  452/  468] time: 61.5084, train_loss: 0.10728654, train_accuracy: 0.9688, test_accuracy: 0.9624
Epoch: [ 0] [  453/  468] time: 61.6280, train_loss: 0.13186580, train_accuracy: 0.9531, test_accuracy: 0.9612
Epoch: [ 0] [  454/  468] time: 61.7525, train_loss: 0.10123964, train_accuracy: 0.9688, test_accuracy: 0.9618
Epoch: [ 0] [  455/  468] time: 61.8898, train_loss: 0.07395607, train_accuracy: 0.9766, test_accuracy: 0.9621
E

In [20]:
# test acc = 96.26